# Snowflake with Ray Air
By using the Ray Snowflake connector to read and write data into and out of Ray Datasets, all of the capabilities of Ray AIR can be used to build end to end machine learning applications. 

## Snowflake with Ray AIR and LightGBM
For this example we will show how to train and tune a [distributed LightGBM](https://docs.ray.io/en/master/ray-air/examples/lightgbm_example.html) model with Ray AIR using Snowflake data. We will then show how to score data with the trained model and push the scored data back into another Snowflake table.

### Set up the connector
The first step is to get a dictionary of connection properties.

In [1]:
import os

# load from environment
env_connect_props = {
    key.replace('SNOWFLAKE_','').lower(): value 
    for key,value in os.environ.items() if 'SNOWFLAKE_' in key
}

# add sample db and schema to connect props
connect_props = {
    **env_connect_props,
    'database':'SNOWFLAKE_SAMPLE_DATA',
    'schema':'TPCH_SF10',
    'warehouse':'COMPUTE_WH'
}

### Training and Tuning
A typical training or tuning workload will have the following logic when working with tabular data in Snowflake:

![Ray Train with Snowflake](images/snowflake_train_with_air.png)

#### Step 1: Stage data in Snowflake
When working with databases, it is best to take advantage of native join and aggregation features of the database prior to ingesting data into Ray Datasets. Ray datasets is designed to power machine learning workflows, and does not provide some typical analytics capabilities like large joins. For these reasons, as a first step, the data required for training the model will be forumated into a single query that runs within Snowflake prior to reading with the Ray Snowflake connector. 
This query could also be materialized into a staging table
if the data neesd to be used repeatedly.

 The code below creates a dataset of customer returns data from several Snowflake sample tables. We will use this data throughtout the train, tune and scoring process. In the code below, we use the `read_snowflake` method to read the data.

> Note: The data set size is set to be small to keep execution times small. If you would like to try larger dataset size, increase the `SIZE` and be sure to have a large enough cluster defined.

In [2]:
SRC = 'SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL'
SIZE = 1000
query = f"""
    WITH cstmrs as (
        SELECT 
            c_customer_sk as c_customer_sk, 
            c_current_cdemo_sk as c_current_cdemo_sk
        FROM {SRC}.customer LIMIT {SIZE}),
    sales as (
        SELECT 
            c_customer_sk, 
            COUNT(c_customer_sk) as n_sales 
            FROM cstmrs JOIN {SRC}.store_sales ON c_customer_sk = ss_customer_sk
        GROUP BY c_customer_sk),
                    
    rtrns as (
        SELECT 
            c_customer_sk, 
            COUNT(c_customer_sk) as n_returns 
            FROM cstmrs JOIN {SRC}.store_returns ON c_customer_sk = sr_customer_sk
        GROUP BY c_customer_sk)
                        
    SELECT
        cstmrs.c_customer_sk as customer_sk,
        ZEROIFNULL(n_sales) as n_sales,
        ZEROIFNULL(n_returns) as n_returns,
        IFF(n_sales is null or n_sales = 0 or n_returns is null, 0, n_returns/n_sales) as return_probability,
        demos.* 
    FROM cstmrs 
    JOIN {SRC}.customer_demographics as demos ON cstmrs.c_current_cdemo_sk = demos.cd_demo_sk
    LEFT OUTER JOIN sales on cstmrs.c_customer_sk = sales.c_customer_sk
    LEFT OUTER JOIN rtrns on cstmrs.c_customer_sk = rtrns.c_customer_sk
"""

#### Step 2: Read data into Ray
Now that we have a qujery, we can read it into a Ray dataset. 
> Note: Only the first partition is read into the dataset. Whe training starts, additional partitions will be pulled into the dataset when needed.

In [3]:
from ray.data import read_snowflake
ds = read_snowflake(connect_props, query=query)
    
ds.limit(10).to_pandas()

2023-02-13 17:01:48,293	INFO worker.py:1242 -- Using address localhost:9031 set in the environment variable RAY_ADDRESS
find: ‘.git’: No such file or directory
2023-02-13 17:01:48,573	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 10.0.0.196:9031...
2023-02-13 17:01:48,578	INFO worker.py:1544 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_vnmb5jgl4z6q98h61dx25rccju/services?redirect_to=dashboard 
2023-02-13 17:01:48,583	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_6097d7d48ea8ab842da11ccf2f0af9f4.zip' (0.79MiB) to Ray cluster...
2023-02-13 17:01:48,593	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_6097d7d48ea8ab842da11ccf2f0af9f4.zip'.
2023-02-13 17:01:53,976	WARNING read_api.py:333 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)

,CUSTOMER_SK,N_SALES,N_RETURNS,RETURN_PROBABILITY,CD_DEMO_SK,CD_GENDER,CD_MARITAL_STATUS,CD_EDUCATION_STATUS,CD_PURCHASE_ESTIMATE,CD_CREDIT_RATING,CD_DEP_COUNT,CD_DEP_EMPLOYED_COUNT,CD_DEP_COLLEGE_COUNT
0,11063183,402,51,0.126866,718589,M,U,2 yr Degree,3000,Low Risk,2,4,2
1,12361894,370,46,0.124324,969956,F,D,2 yr Degree,8500,Good,5,3,3
2,11063159,366,28,0.076503,244699,M,U,4 yr Degree,8000,High Risk,1,6,0
3,11063217,389,40,0.102828,1157747,M,W,Secondary,10000,High Risk,3,1,4
4,11063722,392,52,0.132653,1288460,F,U,2 yr Degree,3500,Good,6,4,4
5,11063419,360,38,0.105556,977178,F,W,4 yr Degree,10000,Low Risk,6,3,3
6,11063765,393,42,0.106870,759334,F,S,4 yr Degree,4000,High Risk,2,5,2
7,11063626,367,33,0.089918,122075,M,D,Unknown,2000,Unknown,0,3,0
8,11063300,367,80,0.217984,250398,F,W,Primary,9000,High Risk,2,6,0
9,11063339,371,51,0.137466,1072234,F,S,4 yr Degree,9000,Low Risk,2,6,3


#### Step 3: Train
Now that the data is read into a Ray dataset, we can use it to train or tune a LighGBM model. 

##### Prepare the data
After reading the data, we need to do some simple manipulations to drop columns and split the data into training and test sets.

In [ ]:
DROP_COLUMNS = ['N_SALES', 'N_RETURNS', 'CD_DEMO_SK']

ds = ds.fully_executed().drop_columns(DROP_COLUMNS).repartition(100)
train_dataset, valid_dataset = ds.train_test_split(test_size=0.3)

##### Create preprocessors
In Ray Air, all trainers, tuners and predcitors allow for the addition of preprocessors. Preprocessors help to featurize data, by providing common operations like on-hot-encoding, categorizing, scaling, etc. For more on the available preprocessors, read the [RayAIR docs](https://docs.ray.io/en/latest/ray-air/package-ref.html#preprocessor). The code below will use a chain of pre-processors. The `BatchMapper` will drop the ID column so it wont be used when training. The `Categorizer` will categorize columns, and the `StandardScaler` will scale columns. All of the pre-processing logic only modifes the data as it is being passed into training algorithms, and the underlying dataset will remain the same.

In [5]:
from ray.data.preprocessors import Chain, BatchMapper, Categorizer, StandardScaler

ID_COLUMN = 'CUSTOMER_SK'
CATEGORICAL_COLUMNS = ['CD_GENDER', 'CD_MARITAL_STATUS', 'CD_EDUCATION_STATUS', 'CD_CREDIT_RATING']
SCALAR_COLUMNS = ['CD_PURCHASE_ESTIMATE', 'CD_DEP_COUNT', 'CD_DEP_EMPLOYED_COUNT', 'CD_DEP_COLLEGE_COUNT']

# Scale some random columns, and categorify the categorical_column,
# allowing LightGBM to use its built-in categorical feature support
preprocessor = Chain(
    BatchMapper(lambda df: df.drop(ID_COLUMN, axis=1), 'pandas'),
    Categorizer(CATEGORICAL_COLUMNS), 
    StandardScaler(columns=SCALAR_COLUMNS)
)

##### Configure scaling
Training requires compute infrastructure, and specifying what type is needed to optimize your training time and costs. When first beginning, it is best to start with a small dataset size and compute to get things working and then scale up data and compute together. Below we create a `ScalingConfig` that provides 10 workers for distributed trianing. This will likely keep training on a single instance. We also don't request GPU's.

In [6]:
from ray.air.config import ScalingConfig

scaling_config=ScalingConfig(num_workers=10, use_gpu=False),

##### Create a trainer
Now that we have everything required, we can create a trainer. In Ray AIR, the logic to create a trainer and fit it are very simliar. The main differences are in the parameters passed to the algorithm. This makes it easy to swap out algorithms. For example, swapping LightGBM for XGBoost, or even PyTorch tabular, will typically be just a few lines of code.

In [7]:
from ray.train.lightgbm import LightGBMTrainer

TARGET_COLUMN = 'RETURN_PROBABILITY'

# LightGBM specific params
params = {
    "objective": "regression",
    "metric": ["rmse", "mae"],
}

trainer = LightGBMTrainer(
    scaling_config=ScalingConfig(num_workers=2, use_gpu=False),
    label_column=TARGET_COLUMN,
    params=params,
    datasets={"train": train_dataset, "valid": valid_dataset},
    preprocessor=preprocessor,
    num_boost_round=10
)

##### Fit the model
Now that the trainer is defined, al that is required is to call fit to begin the training process. The `fit` method will return a results object that containes the model checkpoint as well as model training metrics.

In [ ]:
result = trainer.fit()

(autoscaler +1m16s) Adding 4 node(s) of type worker-node-type-0.


### Score a model
Once there is a trained model, we can use it to score data. The flow for training and scoring are similar in that data is staged in Snowflake and read into a Ray dataset with the connector. Once the data is read in, the previously created model checkpoint can be used to creat a batch predictor for scoring. Scored data can then be written back into Snowflake with the connector. 

The typical logical flow for a batch scoring in Snowflake with Ray AIR is the following:

![Snowflake batch scoring](images/snowflake_score_with_air.png)

#### Steps 1-2: Stage and read data
Since the data has already been staged and loaded, we dont need any extra code to do that now. Typically, you will have a script for training, and a script for scoring that will be run independently. The staging and loading of data should be sperated into a shared script that can be used by each of these workflows.

#### Step 3: Score the data
The previously trained checkpoint can be used to create a predictor. This predictor will already contain the pre-processors used to train the model. All that is needed is to drop the target column before feeding it into the model to simulate a real dataset where we dont know the results.

> Note: Typically model checkpoints will be stored in a model registry provided by Weights and Biases or MLFlow, or into an objects store like S3. Checkpoints are written and read using the [checkpoint API](https://docs.ray.io/en/latest/ray-air/package-ref.html#ray.air.checkpoint.Checkpoint).

In [ ]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.lightgbm import LightGBMPredictor

predictor = BatchPredictor.from_checkpoint(
    result.checkpoint, LightGBMPredictor
)

test_dataset = valid_dataset.drop_columns(TARGET_COLUMN)
predictions = predictor.predict(test_dataset, keep_columns=[ID_COLUMN])
predictions.limit(10).to_pandas()

#### Step 4: Write data to Snowflake
Now that we have the predictions we can write them into a Snowflake table. We need to first create a destination database.

In [ ]:
from ray.data.datasource import SnowflakeConnector

# get new connect proeprties for the new database
write_connect_props = {
    **connect_props, 
    'database':'RAY_SAMPLE', 
    'schema':'PUBLIC'
}

# create destination database
with SnowflakeConnector(**write_connect_props) as con:
    con.query(f'CREATE DATABASE IF NOT EXISTS RAY_SAMPLE')
    

# write the predictions
ds.write_snowflake(
    write_connect_props, 
    table='PREDICTIONS',
    autocreate=True
)

# read the predictions back
read_snowflake(
    write_connect_props, 
    table='PREDICTIONS'
).limit(3).to_pandas()